# Week 12: Capstone System - SOLUTION

Complete production-ready AI agent platform implementation.

In [ ]:
from typing import List, Dict, Optional
import uuid
import time
import asyncio

class AgentTask:
    def __init__(self, description: str, task_type: str = 'general', priority: int = 2):
        self.task_id = str(uuid.uuid4())
        self.description = description
        self.task_type = task_type
        self.priority = priority
        self.status = 'queued'
        self.result = None
        self.created_at = time.time()

class AgentWorker:
    def __init__(self, worker_id: str, capabilities: List[str]):
        self.worker_id = worker_id
        self.capabilities = capabilities
        self.is_busy = False
        self.tasks_completed = 0
    
    async def execute_task(self, task: AgentTask) -> Dict:
        self.is_busy = True
        print(f'Worker {self.worker_id} executing task {task.task_id}')
        await asyncio.sleep(1)  # Simulate work
        self.tasks_completed += 1
        self.is_busy = False
        return {'success': True, 'result': f'Task {task.description} completed'}

class AgentOrchestrator:
    def __init__(self, num_workers: int = 5):
        self.workers = {}
        for i in range(num_workers):
            worker_id = f'worker-{i}'
            self.workers[worker_id] = AgentWorker(worker_id, ['general', 'research'])
        self.task_queue = []
        self.active_tasks = {}
    
    def find_available_worker(self) -> Optional[AgentWorker]:
        for worker in self.workers.values():
            if not worker.is_busy:
                return worker
        return None
    
    async def assign_task(self, task: AgentTask) -> str:
        worker = self.find_available_worker()
        if worker:
            task.status = 'executing'
            self.active_tasks[task.task_id] = task
            result = await worker.execute_task(task)
            task.status = 'completed'
            task.result = result
            return task.task_id
        else:
            self.task_queue.append(task)
            return task.task_id

class ProductionAgentPlatform:
    def __init__(self):
        self.orchestrator = AgentOrchestrator(num_workers=5)
        self.start_time = time.time()
    
    async def submit_task(self, description: str, task_type: str = 'general') -> str:
        task = AgentTask(description, task_type)
        task_id = await self.orchestrator.assign_task(task)
        return task_id
    
    def get_status(self) -> Dict:
        return {
            'uptime_seconds': time.time() - self.start_time,
            'active_tasks': len(self.orchestrator.active_tasks),
            'queued_tasks': len(self.orchestrator.task_queue),
            'total_workers': len(self.orchestrator.workers),
            'busy_workers': sum(1 for w in self.orchestrator.workers.values() if w.is_busy)
        }

print('Production AI Agent Platform - Solution Complete!')
print('Run the platform with: platform = ProductionAgentPlatform()')
print('Submit tasks with: await platform.submit_task("Your task description")')